# Cats vs. Dogs
## Building an image classification model using very little data  

Adapted from https://github.com/rajshah4/image_keras

Based on the tutorial by Francois Chollet @fchollet https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html and the workbook by Guillaume Dominici https://github.com/gggdominici/keras-workshop

This tutorial presents a simple image classifier using keras from just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:  

- training a small network from scratch (as a baseline)  
- using the bottleneck features of a pre-trained network
- fine-tuning the top layers of a pre-trained network

## Data

Data can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data 

All you need is the train set  
The recommended folder structure is:  

### Folder structure

```python
data/
    train/
        dogs/ ### 1024 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 1024 pictures
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/ ### 416 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 416 pictures
            cat001.jpg
            cat002.jpg
            ...
```
Note : for this example we only consider 2x1000 training images and 2x400 testing images among the 2x12500 available.

The github repo includes about 1500 images for this model. The original Kaggle dataset is much larger. The purpose of this demo is to show how you can build models with smaller size datasets. You should be able to improve this model by using more data.

### Data loading

In [1]:
from __future__ import print_function, division
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

Using TensorFlow backend.


In [ ]:
# dimensions of our images
img_width, img_height = 150, 150
batch_size = 32

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

### Imports

In [ ]:
##preprocessing
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
train_generator.class_indices

## Small Conv Net

### Model architecture definition

In [ ]:
# a simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers.
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height, 3))) # Third channel of input is rgb
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # Reduce size of each filter map by half

model.add(Convolution2D(32, (3, 3))) # Kernel size 3x3
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())                # Flatten filter maps from 17x17x64 -> 18496
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))                 # Single neuron for output layer
model.add(Activation('sigmoid'))    # Sigmoid activation squashes output to [0,1]

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Training

In [ ]:
epochs = 30
train_samples = 2048
validation_samples = 832

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples / batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples / batch_size,)
#About 60 seconds an epoch when using CPU

In [ ]:
model.save_weights('models/basic_cnn_30_epochs.h5')

If your model successfully runs at one epoch, go back and it for 30 epochs by changing nb_epoch above.  I was able to get to an val_acc of 0.71 at 30 epochs.

In [ ]:
model.load_weights('models/basic_cnn_30_epochs.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [ ]:
print ("Loss, Accuracy")
model.evaluate_generator(validation_generator, validation_samples)

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_no_dataaugmentation.png)

**After ~10 epochs the neural network reaches ~70% accuracy. We can witness overfitting, no progress is made over validation set in the next epochs**

## Data augmentation for improving the model

By applying random transformation to our train set, we artificially enhance our dataset with new unseen images.  
This will hopefully reduce overfitting and allows better generalization capability for our network.

Example of data augmentation applied to a picture:
![Example of data augmentation applied to a picture](pictures/cat_data_augmentation.png)

In [ ]:
train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,        # normalize pixel values to [0,1]
        shear_range=0.2,       # randomly applies shearing transformation
        zoom_range=0.2,        # randomly applies shearing transformation
        horizontal_flip=True)  # randomly flip the images

# same code as before
train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
model.fit_generator(
        train_generator_augmented,
        steps_per_epoch=train_samples / batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples / batch_size,)

In [ ]:
model.save_weights('models/augmented_30_epochs.h5')

In [ ]:
model.load_weights('models/augmented_30_epochs.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [ ]:
print ("Loss, Accuracy")
model.evaluate_generator(validation_generator, validation_samples)

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 100 epochs :

![Accuracy evolution](pictures/scores_with_dataaugmentation.png)

**Thanks to data-augmentation, the accuracy on the validation set improved to ~80%**

## Using a pre-trained model as a feature extractor

The process of training a convolutional neural network can be very time-consuming and require a significant amount of data.  

We can go beyond the previous models in terms of performance and efficiency by using a general-purpose, pre-trained image classifier.  This example uses VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories. 

On top of it, we add a small multi-layer perceptron and we train it on our dataset.

### VGG16 + small MLP
![VGG16 + Dense layers Schema](pictures/vgg16_original.png)

#### VGG16 model is available in Keras

In [ ]:
model_vgg = applications.VGG16(include_top=False, weights='imagenet')

### Using the VGG16 model to process samples

In [ ]:
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

This is a long process, so we save the output of the VGG16 once and for all.  

In [ ]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, train_samples // batch_size)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [ ]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, validation_samples // batch_size)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

Now we can load it...

In [ ]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (train_samples // 2) + [1] * (train_samples // 2))

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (validation_samples // 2) + [1] * (validation_samples // 2))

And define and train the custom fully connected neural network :

In [ ]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_top.summary()

In [ ]:
model_top.fit(train_data, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_data, validation_labels))

The training process of this small neural network is very fast : ~2s per epoch

In [ ]:
model_top.save_weights('models/bottleneck_30_epochs.h5')

### Bottleneck model evaluation

In [ ]:
model_top.load_weights('models/bottleneck_30_epochs.h5')

Loss and accuracy :

In [ ]:
print ("Loss, Accuracy")
model_top.evaluate(validation_data, validation_labels)

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_with_bottleneck.png)

**We reached a 90% accuracy on the validation after ~1m of training (~20 epochs) and 8% of the samples originally available on the Kaggle competition !**

## Fine-tuning the top layers of a a pre-trained network

Start by instantiating the VGG base and loading its weights.

In [ ]:
model_vgg = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

Build a classifier model to put on top of the convolutional model. For the fine tuning, we start with a fully trained-classifer. We will use the weights from the earlier model. And then we will add this model on top of the convolutional base.

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights('models/bottleneck_30_epochs.h5')

#model_vgg.add(top_model)
model = Model(inputs = model_vgg.input, outputs = top_model(model_vgg.output))

In [ ]:
model.summary()

For fine tuning, we only want to train a few layers.  This line will set the first 15 layers (up to the conv block) to non-trainable.

In [ ]:
for layer in model_vgg.layers[:15]:
    layer.trainable = False

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples / batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples / batch_size)

In [ ]:
model.save_weights('models/finetuning_30epochs_vgg.h5')

In [ ]:
model.load_weights('models/finetuning_30epochs_vgg.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [ ]:
print ("Loss, Accuracy")
model.evaluate_generator(validation_generator, validation_samples)


### Evaluate test set

In [ ]:
from IPython.display import Image
gilbert_file = 'data/validation/cats/gilbert.jpg'
Image(gilbert_file)

In [ ]:
kenji_file = 'data/validation/dogs/kenji.jpg'
Image(kenji_file)

### Resize to 150x150 and stack into a batch

In [ ]:
from scipy import misc
gilbert = misc.imread(gilbert_file)
gilbert = misc.imresize(gilbert, size=(150,150))
gilbert = np.expand_dims(gilbert, 0)

In [ ]:
kenji = misc.imread(kenji_file)
kenji = misc.imresize(kenji, size=(150,150))
kenji = np.expand_dims(kenji, 0)

In [ ]:
batch = np.vstack([gilbert, kenji])
batch.shape

In [ ]:
train_generator.class_indices

### Predict

In [ ]:
model.predict(batch)  # Predict probabilities for the two images. Prob <=0.5 is class cat, >0.5 is class dog